### Instalacion de librerias

In [ ]:
#!pip install yfinance
#!pip install mysql-connector-python

### Imports de librerias

In [3]:
import yfinance as yf
import mysql.connector
from mysql.connector import Error
from datetime import date,datetime,timedelta
import pandas as pd

### Seteo de variables

In [7]:
# diccionario de empresas agrupadas por sector
empresas = {
    'Technology': ['NVDA', 'MSFT', 'ORCL'],
    'Healthcare': ['JNJ', 'PFE', 'ABBV'],
    'Financials': ['AXP', 'JPM', 'BAC']
}

price_data = {}
financials_data = {}
error_log = []

### Funciones para la ingesta de datos en MySQL

In [8]:
def connect_db():
    """
    Establece la conexión a la base de datos stock_market_data.

    Returns:
        connection (mysql.connector.connection.MySQLConnection): conexión activa a MySQL.
    """
    try:
        connection = mysql.connector.connect(
            host='localhost',      
            user='root',       
            password='root',
            database='stock_market_data'
        )
        if connection.is_connected():
            print('Conexión exitosa a la base de datos')
            return connection
    except Error as e:
        print(f' Error al conectar a MySQL: {e}')
        return None


def insert_companies(connection, empresas_dict):
    """
    Inserta empresas en la tabla companies.

    Args:
        connection: Conexión activa a la base de datos.
        empresas_dict (dict): Diccionario {sector: [tickers]}.
    """
    cursor = connection.cursor()

    for sector, tickers in empresas_dict.items():
        for ticker in tickers:
            try:
                query = """
                INSERT IGNORE INTO companies (ticker, sector)
                VALUES (%s, %s)
                """
                cursor.execute(query, (ticker, sector))
            except Error as e:
                print(f"Error insertando company {ticker}: {e}")

    connection.commit()
    cursor.close()


def insert_stock_prices(connection, price_data):
    """
    Inserta precios históricos en la tabla stock_prices.

    Args:
        connection: Conexión activa a la base de datos.
        price_data (dict): Diccionario {ticker: dataframe de precios}.
    """
    cursor = connection.cursor()

    for ticker, df in price_data.items():
        for _, row in df.iterrows():
            try:
                query = """
                INSERT IGNORE INTO stock_prices (ticker, date, open, high, low, close, volume)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """
                data = (
                    ticker,
                    row['Date'],
                    row['Open'],
                    row['High'],
                    row['Low'],
                    row['Close'],
                    int(row['Volume']) if not pd.isna(row['Volume']) else None
                )
                cursor.execute(query, data)
            except Error as e:
                print(f"Error insertando precio {ticker} en {row['Date']}: {e}")

    connection.commit()
    cursor.close()
    

def insert_financial_metrics(connection, financials_data):
    """
    Inserta métricas financieras en la tabla financial_metrics.

    Args:
        connection: Conexión activa a la base de datos.
        financials_data (dict): Diccionario {ticker: dataframe de métricas financieras}.
    """
    cursor = connection.cursor()

    for ticker, df in financials_data.items():
        for index, row in df.iterrows():
            for column, value in row.items():
                if column != 'Ticker' and pd.notnull(value):
                    try:
                        query = """
                        INSERT IGNORE INTO financial_metrics (ticker, period, metric_name, metric_value)
                        VALUES (%s, %s, %s, %s)
                        """
                        data = (ticker, index.date() if isinstance(index, pd.Timestamp) else index, column, float(value))
                        cursor.execute(query, data)
                    except Error as e:
                        print(f"Error insertando métrica {ticker} {column} en {index}: {e}")

    connection.commit()
    cursor.close()

### Funciones para el control de calidad

In [9]:
def resumen_errores():
    """
    Genera e imprime un resumen de los errores detectados durante el proceso.
    """
    if not error_log:
        print("\n No se detectaron errores en el proceso.")
        return

    df_errores = pd.DataFrame(error_log)
    conteo = df_errores['Error'].value_counts()

    print("\n Resumen de errores:")
    for tipo, cantidad in conteo.items():
        print(f"- {cantidad} errores en {tipo.lower()}")

    print("\n Detalle de errores:")
    print(df_errores)


def data_quality_control(df: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """
    Limpia el DataFrame de precios eliminando columnas irrelevantes,
    gestionando valores faltantes y corrigiendo valores anómalos.

    Args:
        df (pd.DataFrame): DataFrame de precios históricos.
        ticker (str): Ticker de la empresa.

    Returns:
        pd.DataFrame: DataFrame limpio y validado.
    """
    # Eliminar columnas no deseadas
    columns_to_drop = ['Dividends', 'Stock Splits']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

    # Formateo del campo date 
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date']).dt.date

    # Identificar y eliminar valores faltantes
    missing_values = df.isnull().sum()
    if missing_values.any():
        print(f'[{ticker}] Atención: valores faltantes detectados:')
        print(missing_values[missing_values > 0])
        df = df.dropna()

    # Controlar precios y volúmenes negativos
    if (df['Close'] < 0).any():
        print(f'[{ticker}] Error: precios negativos detectados en "Close". Corrigiendo...')
        df = df[df['Close'] >= 0]
    
    if (df['Volume'] < 0).any():
        print(f'[{ticker}] Error: volumen negativo detectado. Corrigiendo...')
        df = df[df['Volume'] >= 0]
    
    return df

### Funciones para la extraccion de datos bursatiles

In [10]:
def get_company_data(ticker: str):
    """
    Descarga precios históricos y métricas financieras para una empresa específica,
    gestionando errores y registrando incidencias.

    Args:
        ticker (str): Código bursátil de la empresa.
    """
    try:
        stock = yf.Ticker(ticker)
        
        # Obtener precios históricos
        try:
            hist = stock.history(period='2y')
            if hist.empty:
                raise ValueError(f"[{ticker}] No se obtuvieron precios históricos.")
            hist.reset_index(inplace=True)
            hist['Ticker'] = ticker
            hist = data_quality_control(hist, ticker)
            price_data[ticker] = hist
        except Exception as e_hist:
            print(f'[{ticker}] Error en precios históricos: {e_hist}')
            error_log.append({'Ticker': ticker, 'Error': 'Precio', 'Mensaje': str(e_hist)})
        
        # Obtener métricas financieras clave
        try:
            fin = stock.financials.T
            bs = stock.balance_sheet.T
            if fin.empty and bs.empty:
                raise ValueError(f"[{ticker}] No se obtuvieron datos financieros.")
            info = pd.concat([fin, bs], axis=1)
            info['Ticker'] = ticker
            financials_data[ticker] = info
        except Exception as e_fin:
            print(f'[{ticker}] Error en datos financieros: {e_fin}')
            error_log.append({'Ticker': ticker, 'Error': 'Financiero', 'Mensaje': str(e_fin)})

    except Exception as e_general:
        print(f'[{ticker}] Error general: {e_general}')
        error_log.append({'Ticker': ticker, 'Error': 'General', 'Mensaje': str(e_general)})

### Programa principal

In [ ]:
if __name__ == "__main__":
    # Extracción de datos para cada empresa
    for sector, tickers in empresas.items():
        for ticker in tickers:
            get_company_data(ticker)

    # Mostrar resumen de errores de extracción
    resumen_errores()

    # Conectar a la base de datos
    conn = connect_db()

    if conn:
        # Insertar datos en MySQL
        insert_companies(conn, empresas)
        insert_stock_prices(conn, price_data)
        insert_financial_metrics(conn, financials_data)

        # Cerrar conexión
        conn.close()
        print(' Datos insertados correctamente y conexión cerrada.')
    else:
        print(' No se pudo conectar a la base de datos. No se insertaron datos.')   


 No se detectaron errores en el proceso.
